<a href="https://colab.research.google.com/github/lcunicorn/Text_generation/blob/master/Copy_of_text_generation_01_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/beta/tutorials/text/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/r2/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/site/en/r2/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to generate text using a character-based RNN. We will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware acclerator > GPU*. If running locally make sure TensorFlow version >= 1.11.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/programmers_guide/keras) and [eager execution](https://www.tensorflow.org/programmers_guide/eager). The following is sample output when the model in this tutorial trained for 30 epochs, and started with the string "Q":

## Setup

### Import TensorFlow and other libraries

In [0]:
!pip install tensorflow-gpu==2.0.0-beta1
!pip install tensorflow==2.0.0-beta1

     |████████████████████████████████| 348.9MB 64kB/s 
     |████████████████████████████████| 501kB 40.1MB/s 
     |████████████████████████████████| 3.1MB 30.9MB/s 
     |████████████████████████████████| 87.9MB 37.5MB/s 
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [0]:
import tensorflow as tf
print(tf.__version__)

2.0.0-beta1


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import os
import time

In [0]:
pip install langdetect

     |████████████████████████████████| 1.0MB 2.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [0]:
# Custom libraries
import pandas as pd
import langdetect
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import re

### Upload the dataset

In [0]:
from google.colab import files
uploaded = files.upload()

Saving Die_Gruenen_n.csv to Die_Gruenen_n.csv


### Preprocess the data (as text)

In [0]:
df = pd.read_csv('Die_Gruenen_n.csv', delimiter=';', engine='python', encoding='utf-8')

In [0]:
# Filter out a dataframe with only German-language tweets
party_de = pd.DataFrame()
for x in range(len(df)):
  if pd.notnull(df.tweets.iloc[x]):
    #y = re.sub('^[a-zA-ZäöüßÄÖÜ]', ' ', df.tweets[x])
    y = df.tweets[x]
    try:
        language = langdetect.detect(y)
    except:
        language = "error"
        print("This row throws and error:", y)
    if language == 'de':
        party_de = party_de.append(df.loc[x])
df = party_de.reset_index(drop=True)

This row throws and error: 11. - 13.11.
This row throws and error: http://m.gruene.de/ueber-uns/gruene-setzen-aufarbeitung-fort.html …
This row throws and error: .@Schmidtlepp https://www.gruene.de/ueber-uns/hier-kannst-du-direkt-mitglied-werden.html …


In [0]:
# Preprocessing function (no lemmatization)
def preprocess_corpus(corpus):
    preprocessed = []
    for tweet in corpus:
        # delete links
        tweet = re.sub('pic.twitter', ' pic.twitter', tweet)
        tweet = re.sub('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', ' ', tweet)
        tweet = re.sub('pic.twitter.*', '', tweet)
        # Delete line breaks, extra spaces, byte order mark
        tweet = re.sub('\ufeff', ' ', tweet)
        tweet = re.sub('[\n\r]', ' ', tweet)
        tweet = re.sub('\\\\n', ' ', tweet)
        tweet = re.sub('\\\\r', ' ', tweet)
        #tweet = re.sub('[^a-zA-ZäüöÄÜÖß0-9@# ]', ' ', tweet)
        tweet = re.sub('\\s+', ' ', tweet)
        '''
        # Lemmatize
        nlp_text = nlp(tweet)
        lemmas = [token.lemma_ for token in nlp_text if token.lemma_ not in STOP_WORDS]
        tweet = ' '.join(lemmas).lower()
        tweet = re.sub('@ ', '@', tweet)
        tweet = re.sub('# ', '#', tweet)        
        '''
        preprocessed.append(tweet)
    return preprocessed

In [0]:
text = preprocess_corpus(df.tweets)
text = " ".join(text)

In [0]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

115 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  '!' :   1,
  '#' :   2,
  '%' :   3,
  '&' :   4,
  "'" :   5,
  '(' :   6,
  ')' :   7,
  '*' :   8,
  '+' :   9,
  ',' :  10,
  '-' :  11,
  '.' :  12,
  '/' :  13,
  '0' :  14,
  '1' :  15,
  '2' :  16,
  '3' :  17,
  '4' :  18,
  '5' :  19,
  ...
}


In [0]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'Heute haben @' ---- characters mapped to int ---- > [38 64 80 79 64  0 67 60 61 64 73  0 30]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [0]:
#examples_per_epoch = list(df.tweets)
#bzw später iwie code so anpassen, dass es sinn macht für eizelne tweets

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 280
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

H
e
u
t
e


The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'Heute haben @jamila_anna und Robert Habeck stellvertretend für den Bundesvorstand gemeinsam mit @ManuelSarrazin den Präsidenten der Ukraine @ZelenskyyUa in unserer Parteizentrale getroffen. Wir haben versichert: Wir stehen an der Seite der Zivilgesellschaft der Ukraine. Robert #Ha'
'beck hat mit @zeitonline über den grünen Erfolg bei der #Europawahl gesprochen. Wie wir den Hunger nach politischer Handlungsfähigkeit stillen und für ökologische Veränderungen kämpfen möchten, erzählt er im Interview: … Hans Christian Ströbele @MdB_Stroebele feiert heute seinen 8'
'0. Geburtstag. Ströbi, leidenschaftlicher Demonstrant, Mann der Straße, da für die Menschen – alles Gute! Lebe hoch!  Das #HauAbGesetz verschlechtert die Situation von Asylsuchenden und erschwert Integration. Wir lehnen diese massive Verschärfung des Asylrechts ab und kämpfen weit'
'er für eine Flüchtlingspolitik, die auf Menschenrechten gründet. Gestern war @ABaerbock bei @maybritillner im @ZDF: TV-Tipp: Heute Abend 22:15 Uhr 

For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
len(text)

619323

In [0]:
sequences

<BatchDataset shapes: (281,), types: tf.int64>

Print the first examples input and target values:

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Heute haben @jamila_anna und Robert Habeck stellvertretend für den Bundesvorstand gemeinsam mit @ManuelSarrazin den Präsidenten der Ukraine @ZelenskyyUa in unserer Parteizentrale getroffen. Wir haben versichert: Wir stehen an der Seite der Zivilgesellschaft der Ukraine. Robert #H'
Target data: 'eute haben @jamila_anna und Robert Habeck stellvertretend für den Bundesvorstand gemeinsam mit @ManuelSarrazin den Präsidenten der Ukraine @ZelenskyyUa in unserer Parteizentrale getroffen. Wir haben versichert: Wir stehen an der Seite der Zivilgesellschaft der Ukraine. Robert #Ha'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 38 ('H')
  expected output: 64 ('e')
Step    1
  input: 64 ('e')
  expected output: 80 ('u')
Step    2
  input: 80 ('u')
  expected output: 79 ('t')
Step    3
  input: 79 ('t')
  expected output: 64 ('e')
Step    4
  input: 64 ('e')
  expected output: 0 (' ')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [0]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 280), (64, 280)), types: (tf.int64, tf.int64)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next character:

![A drawing of the data passing through the model](https://tensorflow.org/tutorials/beta/text/images/text_generation_training.png)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 280, 115) # (batch_size, sequence_length, vocab_size)


In [0]:
dataset.take(1)

<TakeDataset shapes: ((64, 280), (64, 280)), types: (tf.int64, tf.int64)>

In [0]:
test_dataset = dataset.take(1000) 
train_dataset = dataset.skip(1000)

In [0]:
train_dataset

<SkipDataset shapes: ((64, 280), (64, 280)), types: (tf.int64, tf.int64)>

In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           29440     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 115)           117875    
Total params: 5,394,291
Trainable params: 5,394,291
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [0]:
sampled_indices

array([ 91,  63,  82,  97,  62,  62,  98,  77,  18,  15,  82,  71, 113,
        80, 105,  83,  63,  55,  54,  63,  88,   9,  96,  50, 105,   2,
        90,  77,  65,  98,  76,  53,  54,  80,  62,  24,   0, 112,  34,
         4,  25,   7,  21,  54,   7,  33,  97,  70, 105,  67, 103,  88,
        35,  88,   8,  62, 108,   7,  65,  87,   8,  29,  44,  15,   6,
        79,  63,  43, 114, 114,  61, 109,  43,  12,  39,   3,  89,  18,
        23, 107, 110,  65,  64,  24,  87,  97,  79,  83,  64,  13,  36,
        83,  93,  52,  84,  34,   8,  41, 107,  26,  97,  74,  62,  68,
        56,  45,  79,  75,  48, 113,  68,  87,  30,  88,  43,  23,  52,
        67,  80,  73,  72,  29,  29,  61,  86,  23,  82,  51,  96,  37,
        42,  78, 112,  32,  61, 105,   0,  18,  57,  34,  15,  54,  28,
        99,  57,  63,  17,  25,  91,  47,  91,  73,  45,  15,  80,  62,
        22,  93,  28,  10,   1,  26,  77,  63,  51,  58,  65, 103,  88,
        60, 106,  47,  13,  44,  58,  39,  79,  51,  58,  85,  6

Decode these to see the text predicted by this untrained model:

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_softmax_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 280, 115)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.7439084


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [0]:
from keras import metrics

Using TensorFlow backend.


In [0]:
model.compile(optimizer='adam', loss=loss, metrics=[tf.keras.metrics.Accuracy()])

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [0]:
EPOCHS=20

In [0]:
history = model.fit(dataset, validation_data=test_dataset, epochs=EPOCHS, callbacks=[checkpoint_callback], verbose=1)

#history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback], verbose=1)

Epoch 1/20


InvalidArgumentError: ignored

In [0]:
import matplotlib.pyplot as plt

In [0]:
# Plot accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

KeyError: ignored

In [0]:
# Plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

In [0]:
import pickle

In [0]:
#Save the model
model.save('tf_rnnmodel_2706.h5')
#files.download('tf_rnnmodel_2706.h5') 

## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            29440     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 115)            117875    
Total params: 5,394,291
Trainable params: 5,394,291
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](https://tensorflow.org/tutorials/beta/text/images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [0]:
def generate_text(model, start_string, temperature):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 150

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  #temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"Die CDU ", temperature=0.3))

Die CDU bei #AnneWill #LR13 #btw17 @malterschutz #btw17 'Wir wollen die Berlin. Das wird das mit dem #Merkel in der #BDK14 'Wir wollen die #Europawahl gegen #


The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.

So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point if, for example, to implement _curriculum learning_ to help stabilize the model's open-loop output.

We will use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The procedure works as follows:

* First, initialize the RNN state. We do this by calling the `tf.keras.Model.reset_states` method.

* Next, iterate over the dataset (batch by batch) and calculate the *predictions* associated with each.

* Open a `tf.GradientTape`, and calculate the predictions and loss in that context.

* Calculate the gradients of the loss with respect to the model variables using the `tf.GradientTape.grads` method.

* Finally, take a step downwards by using the optimizer's `tf.train.Optimizer.apply_gradients` method.



In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [0]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))